In [7]:
import os
import math

import matplotlib.pyplot as plt 

import numpy as np
import pandas as pd
import random

In [8]:
#global variables
maxT = 150
maxLambda = 720 # there is also min lamda at around 350nm, one can do a better normalization
PAD_VALUE = -1
NEvts = 1000

In [9]:
# takes input array and creates a list of arrays, where each array in the list is a 1D event
# each 1D event is [hit1_theta, hit1_phi, hit1_time, hit1_color,   hit2_theta, hit2_phi, hit2_time, hit2_color etc]
# takes color only if take_color=1
def create_images(data, take_color=1):
    #print(data[:,0])
    images = []
    for i_event in range(0,NEvts) :
        X = data[np.where(data[:,0]==i_event)] #take all rows where 1st colummn correspond to current event
        X = np.delete(X,0,1) #delete column with event number
        
        if take_color==0 :
            X = np.delete(X,3,1) #delete column with color
        
        n_phot = X.shape[0]
        
        #keeping only a pre-set percentage of photons (mimic photo-coverage)
        ind = np.arange(n_phot)
        np.random.shuffle(ind)
        n_phot = int(np.ceil(n_phot*0.65)) # 65% coverage
        X = X[ ind[:n_phot], : ]
        
#         # The commented section below is only needed for time semearing
#         # can be left commented when working with true time
        
#         ##time_jitter = np.random.normal(0,0.4899,n_phot)
#         ##time_jitter = np.random.normal(0,0.7433,n_phot)
#         #time_jitter = np.random.normal(0,0.229129,n_phot)
#         #np.random.shuffle(time_jitter)
#         #time_jitter = time_jitter.reshape(n_phot,1)
#         th_ph = np.zeros((n_phot,2))
#         if take_color==1 :
#             color = np.zeros((n_phot,1))
#         #time_jitter = np.hstack((th_ph,time_jitter))
#         #X = X - time_jitter
        
#         t_min = np.ones((n_phot,1))*min(X[:,2])
#         t_min = np.hstack((th_ph,t_min))
#         if take_color==1 :
#             t_min = np.hstack((t_min,color))
#         X = X - t_min
             
        X=X[np.argsort(X[:,2])]  #order hits by time within an event
     
        
        X[:,0] /= np.pi        #normalizing theta
        X[:,1] /= (2*np.pi)    #normalizing phi
        X[:,2] /= maxT         #normalizing time (relative to the first hit)
        if take_color==1 :
            X[:,3] /= maxLambda       #normalizing color
                                # there is also min lamda at around 350nm, one can do a better normalization
        
        X = X.reshape(X.shape[0]*X.shape[1]) #reshape into 1D array

        images.append(X)
        
    return images

In [10]:
from keras.utils import Sequence
from keras.preprocessing.sequence import pad_sequences

def process_a_folder(input_dir, take_color, out_dir, fn) :
    x1=os.listdir(input_dir)
    x1.sort()
    print(x1)
    print(len(x1))
    for idx in range (0,len(x1)) :
        print(idx)
        print(x1[idx])
        #print(input_dir)
        print(os.path.join(input_dir, x1[idx]))
        batch_data1 = np.load(os.path.join(input_dir, x1[idx]))
        data1 = create_images(batch_data1['evt_theta_phi_time'],take_color)
        vtx1 = batch_data1['vtx_xyz']
        dir1 = batch_data1['dir_xyz']
                
        #longest_row = max(data1,key=len) #find longest row in data
        #print(len(longest_row))
        Lmax = 6300 #len(longest_row) #lenght of the longest row - this is needed for padding
        if take_color==1 :
            Lmax = 8400
        
        batch_x1 = pad_sequences(data1, maxlen=Lmax, dtype='float64', padding='post', truncating='post', value=PAD_VALUE)
        
        out_file = 'f_batch_' + str(int(fn)+idx) + '.npz'
        np.savez(os.path.join(out_dir, out_file), x=batch_x1, y_vtx=vtx1, y_dir=dir1)
        
    return 0

In [11]:
# from keras.utils import Sequence
# from keras.preprocessing.sequence import pad_sequences

# def process_a_folder(input_dir, take_color, out_dir, fn) :
#     x1=os.listdir(input_dir)
#     x1.sort()
#     print(x1)
#     print(len(x1))
#     for idx in range (0,len(x1)) :
#         print(idx)
#         print(x1[idx])
#         #print(input_dir)
#         print(os.path.join(input_dir, x1[idx]))
#         batch_data1 = np.load(os.path.join(input_dir, x1[idx]))
#         data1 = create_images(batch_data1['evt_theta_phi_time'],take_color)
#         vtx1 = batch_data1['vtx_xyz']
#         dir1 = batch_data1['dir_xyz']
                
#         #longest_row = max(data1,key=len) #find longest row in data
#         #print(len(longest_row))
#         Lmax = 6300 #len(longest_row) #lenght of the longest row - this is needed for padding
        
#         batch_x1 = pad_sequences(data1, maxlen=Lmax, dtype='float64', padding='post', truncating='post', value=PAD_VALUE)
#         batch_y1 = np.hstack((vtx1,dir1))
        
#         #the following is only needed for shuffling, not clear if that's needed for vertexing
#         batch = np.concatenate((batch_x1, batch_y1), axis=1)
#         batch_x1, batch_y1 = [], []
#         np.random.shuffle(batch)
#         batch_y1 = batch[:,-6:]
#         batch_y1 = np.delete(batch_y1, np.s_[-3:],1) ### for the time being deleting directionality info
        
#         batch_x1 = np.delete(batch, np.s_[-6:],1)
#         ###  end of the shuffling segment
        
#         print('batch_x1.shape = ',batch_x1.shape, 'batch_y1.shape = ',batch_y1.shape)
#         #print(batch_x)
#         #print(batch_y)
#         print('=====')
#         print('')
#         batch = []
     
        
#         out_file = 'f_batch_' + str(int(fn)+idx+1) + '.npz'
#         np.savez(os.path.join(out_dir, out_file), x=batch_x1, y=batch_y1)
        
#     return 0

In [ ]:
process_a_folder('/data/Elagin/vtx_dir_1el_sciLight_16dd/', 0, '/data/Elagin/vtx_dir_1el_sciLight_norm',1600)
#process_a_folder('/data/Elagin/data_vtx_dir_1el_test/', 0, '/data/Elagin/data_vtx_dir_1el_test_norm',1)
#process_a_folder('/data/Elagin/sig_test_maxT52_che', '/data/Elagin/bkg_test_maxT52_che', 0, '/data/Elagin/test_maxT52_che')

['photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1600.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1601.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1602.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1603.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1604.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1605.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1606.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1607.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_1608.npz', 'photon_data_with_vtx_dir_xyz_sciLight_maxTall_1el_2p529MeV_dVrndVtx_3p0mSphere_rndDir_1k_

In [14]:
!ls /data/Elagin/vtx_dir_1el_sciLight_norm

f_batch_1500.npz  f_batch_1525.npz  f_batch_1550.npz  f_batch_1575.npz
f_batch_1501.npz  f_batch_1526.npz  f_batch_1551.npz  f_batch_1576.npz
f_batch_1502.npz  f_batch_1527.npz  f_batch_1552.npz  f_batch_1577.npz
f_batch_1503.npz  f_batch_1528.npz  f_batch_1553.npz  f_batch_1578.npz
f_batch_1504.npz  f_batch_1529.npz  f_batch_1554.npz  f_batch_1579.npz
f_batch_1505.npz  f_batch_1530.npz  f_batch_1555.npz  f_batch_1580.npz
f_batch_1506.npz  f_batch_1531.npz  f_batch_1556.npz  f_batch_1581.npz
f_batch_1507.npz  f_batch_1532.npz  f_batch_1557.npz  f_batch_1582.npz
f_batch_1508.npz  f_batch_1533.npz  f_batch_1558.npz  f_batch_1583.npz
f_batch_1509.npz  f_batch_1534.npz  f_batch_1559.npz  f_batch_1584.npz
f_batch_1510.npz  f_batch_1535.npz  f_batch_1560.npz  f_batch_1585.npz
f_batch_1511.npz  f_batch_1536.npz  f_batch_1561.npz  f_batch_1586.npz
f_batch_1512.npz  f_batch_1537.npz  f_batch_1562.npz  f_batch_1587.npz
f_batch_1513.npz  f_batch_1538.npz  f_batch_1563.npz  f_batch_1588.npz
f_batc

In [ ]:
batch = np.load('/data/Elagin/photon_data_with_vtx_dir_allLight_maxT37p0ns_1el_2p529MeV_dVrndVtx_0p5mSphere_rndDir_1k_1.npz')
#print(batch['evt_theta_phi_time'])
print(batch['vtx_theta_phi_r'])
print(batch['dir_theta_phi'])

print(np.hstack((batch['vtx_theta_phi_r'],batch['dir_theta_phi'])))


In [ ]:
dir1 = '/data/Elagin/sig_test_true_time_maxT37/'
x_list = os.listdir(dir1)
x_list.sort() #alphanumerical sorting
print(x_list[0])
batch_data1 = np.load(os.path.join(dir1,x_list[0]))
data1 = create_images(batch_data1['evt_theta_phi_time'],1)


In [ ]:
print(len(data1[0]))
print(data1[0])

longest_row = max(data1,key=len) #find longest row in data
print(len(longest_row))
